<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [973]:
import pandas as pd
import psycopg2

In [975]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

print(connection)

<connection object at 0x11a094820; dsn: 'user=skillfactory password=xxx dbname=project_sql host=84.201.134.129 port=5432', closed: 0>


# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [976]:
# Запрашиваем общее количество вакансий в базе данных.
# Это даст нам представление о размере выборки для дальнейшего анализа.
query = f'''
SELECT count(*)
FROM public.vacancies
'''

In [977]:
df = pd.read_sql_query(query, connection)
# Выведем результат запроса в виде таблицы.
# Это позволит быстро увидеть, сколько всего вакансий в базе.
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/2678056138.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [978]:
# Запрос для подсчёта количества работодателей в базе данных.
# Это поможет понять, сколько уникальных компаний размещают вакансии на платформе.
query = f'''
select count(*)
from public.employers
'''

In [979]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [980]:
# Формируем SQL-запрос для подсчёта количества регионов в базе данных.
# Здесь мы обращаемся к таблице public.areas и считаем все строки — это и есть регионы.
query = f'''
select count(*)
from public.areas
'''

In [981]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [982]:
# Формируем SQL-запрос для подсчёта количества сфер деятельности в базе.
# Здесь мы обращаемся к таблице public.industries и считаем все строки — это и есть сферы деятельности.
query = f'''
select count(id)
from public.industries
'''

In [983]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,294


***

## Основные метрики базы:

| Показатель                  | Полученные значения   |
|-----------------------------|------------|
| Количество вакансий         | **49 197** |
| Количество работодателей    | **23 501** |
| Количество регионов         | **1 362**  |
| Количество сфер деятельности| **294**    |

## Выводы:

### Количество вакансий — **49 197**
- Большой объём данных.
- Хорошая основа для устойчивой и обобщающей модели машинного обучения.

### Количество работодателей — **23 501**
- Примерно **2,1 вакансии на одного работодателя**.
- Высокое разнообразие компаний.
- Модель сможет учитывать предпочтения пользователей по типу компаний.

### Количество регионов — **1 362**
- Широкая географическая представленность.
- Географический фактор можно включить в модель как важный признак.

### Количество сфер деятельности — **294**
- Высокая предметная разнородность.
- Возможность учитывать индустриальные предпочтения клиентов (например, финансы, медицина, IT и т.д.).
- Важно для фильтрации и тонкой настройки рекомендаций даже внутри профессии *Data Scientist*.

## Общий вывод:

- Данные **объемные, разнообразные и структурно пригодные** для построения рекомендательной системы.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [984]:
# Формируем SQL-запрос для получения топ-5 регионов по количеству вакансий.
# 1. Считаем количество вакансий (count(vac.id)) для каждого региона (ar.name).
# 2. Объединяем таблицы вакансий и регионов по идентификатору региона.
# 3. Группируем результат по названию региона.
# 4. Сортируем по количеству вакансий в убывающем порядке.
# 5. Оставляем только первые 5 строк — это и будут регионы-лидеры по вакансиям.

query = f'''
select
	count(vac.id) as cnt,      -- Считаем вакансии в регионе
	ar.name as area            -- Название региона
from public.vacancies as vac
join areas as ar
on vac.area_id = ar.id         -- Соединяем вакансии с регионами
group by ar.name               -- Группируем по региону
order by cnt desc              -- Сортируем по убыванию количества
limit 5                        -- Берём только топ-5
'''

In [985]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt,area
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [986]:
# SQL-запрос для подсчёта количества вакансий, где указана хотя бы одна из границ зарплаты
query = f'''
select
	count(vac.id)  -- Считаем количество вакансий (id) в таблице vacancies
from public.vacancies as vac
where salary_to is not null or salary_from is not null  -- Оставляем только те вакансии, где заполнено хотя бы одно поле зарплаты
'''
# Здесь мы просто хотим узнать, сколько вакансий содержат информацию о зарплате — хотя бы минимальную или максимальную.

In [987]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [988]:
# SQL-запрос для подсчёта средних значений нижней и верхней границы зарплатной вилки по всем вакансиям.
# round(avg(vac.salary_from)) — вычисляет среднее значение по минимальной зарплате и округляет до целого.
# round(avg(vac.salary_to)) — вычисляет среднее значение по максимальной зарплате и тоже округляет.
# Всё это берётся из таблицы вакансий public.vacancies.

query = f'''
select
	round(avg(vac.salary_from)),  -- Средняя нижняя граница зарплаты (округлённая)
	round(avg(vac.salary_to))     -- Средняя верхняя граница зарплаты (округлённая)
from public.vacancies as vac
'''

In [989]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,round,round
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.

In [990]:
# Формируем SQL-запрос для анализа сочетаний графика работы и типа занятости в вакансиях.
# В первой строке мы объединяем значения schedule (график работы) и employment (тип занятости) в одну строку через пробел — это и будет наше уникальное сочетание.
# Затем считаем, сколько раз встречается каждое такое сочетание (count(*)).
# Данные берём из таблицы public.vacancies.
# Группируем по нашему новому полю comb — чтобы получить статистику по каждому варианту.
# Сортируем результат по убыванию количества — самые популярные сочетания будут вверху.

query = f'''
select
	vac1.schedule || ' ' || vac1.employment as comb,  -- объединяем график и занятость в одну строку
	count(*) as cnt                                   -- считаем количество таких вакансий
from 
	public.vacancies as vac1
group by comb                                         -- группируем по сочетанию
order by cnt desc                                     -- сортируем по убыванию
'''

In [991]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,comb,cnt
0,Полный день Полная занятость,35367
1,Удаленная работа Полная занятость,7802
2,Гибкий график Полная занятость,1593
3,Удаленная работа Частичная занятость,1312
4,Сменный график Полная занятость,940
5,Полный день Стажировка,569
6,Вахтовый метод Полная занятость,367
7,Полный день Частичная занятость,347
8,Гибкий график Частичная занятость,312
9,Полный день Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [992]:
# Формируем SQL-запрос для анализа распределения вакансий по требуемому опыту.
# В первой строке выбираем поле experience (требуемый опыт) из таблицы вакансий.
# Считаем количество вакансий для каждого варианта опыта (count(*)).
# Группируем результат по опыту — так получим, сколько вакансий для каждого уровня.
# Сортируем по количеству вакансий в порядке возрастания (order by 2).
query = f'''
select 
	vac.experience as experience,  -- Берём вариант опыта из вакансии
	count(*)                      -- Считаем, сколько таких вакансий
from public.vacancies as vac
group by experience              -- Группируем по опыту
order by 2                       -- Сортируем по количеству (по второй колонке)
'''

In [993]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

## Аналитический отчет по вакансиям hh.ru

## 1. Распределение вакансий по регионам

| Вакансий | Регион           |
|----------|------------------|
| 5 333    | Москва           |
| 2 851    | Санкт-Петербург  |
| 2 112    | Минск            |
| 2 006    | Новосибирск      |
| 1 892    | Алматы           |

**Вывод:**
- Наибольшее количество вакансий сосредоточено в крупных городах: Москве и Санкт-Петербурге.
- Также в топ-5 попадают крупные региональные и международные центры (Минск, Алматы, Новосибирск).
- Модель рекомендаций должна учитывать высокую концентрацию вакансий в мегаполисах при формировании предложений пользователям.

---

## 2. Количество вакансий с заполненной хотя бы одной границей зарплаты

| Вакансий с указанной зарплатой |
|--------------------------------|
| 24 073                         |

**Вывод:**
- Около 49% (из 49 197 вакансий) содержат информацию хотя бы об одной границе зарплатной вилки.
- Эти данные можно использовать как таргет или важный признак в модели, при этом стоит учитывать, что почти половина вакансий не содержит зарплаты, и нужно будет обрабатывать пропуски.

---

## 3. Средние значения зарплатных вилок

| Средняя нижняя граница | Средняя верхняя граница |
|------------------------|--------------------------|
| 71 065 ₽               | 110 537 ₽                |

**Вывод:**
- Средняя зарплатная вилка для вакансий составляет примерно **71K–110K рублей**.
- Это может быть полезно для расчёта рыночных ориентиров при сравнении вакансий или построении рекомендательных правил.
- Зарплатные ожидания пользователя могут сравниваться с этими диапазонами для фильтрации или ранжирования вакансий.

---

## 4. Распределение вакансий по типу графика и трудоустройства

| Сочетание графика и занятости                  | Вакансий |
|------------------------------------------------|----------|
| Полный день Полная занятость                   | 35 367   |
| Удаленная работа Полная занятость              | 7 802    |
| Гибкий график Полная занятость                 | 1 593    |
| Удаленная работа Частичная занятость           | 1 312    |
| Сменный график Полная занятость                | 940      |
| Полный день Стажировка                         | 569      |
| Вахтовый метод Полная занятость                | 367      |
| Полный день Частичная занятость                | 347      |
| Гибкий график Частичная занятость              | 312      |
| Полный день Проектная работа                   | 141      |
| Удаленная работа Проектная работа              | 133      |
| Гибкий график Стажировка                       | 116      |
| Сменный график Частичная занятость             | 101      |
| Удаленная работа Стажировка                    | 64       |
| Гибкий график Проектная работа                 | 18       |
| Сменный график Стажировка                      | 12       |
| Вахтовый метод Проектная работа                | 2        |
| Сменный график Проектная работа                | 1        |

**Вывод:**
- Наиболее популярный формат: **Полный день + Полная занятость** (72% от всех вакансий).
- Удалённая работа и гибкий график уверенно занимают вторые позиции, особенно в сочетании с полной или частичной занятостью.
- Это подтверждает тренд на гибкость условий труда, особенно для профессий, связанных с IT.
- В модели стоит учитывать предпочтения по типу графика, особенно если пользователь указывает требования по удалёнке.

---

## 5. Распределение вакансий по требуемому опыту

| Требуемый опыт          | Вакансий |
|-------------------------|----------|
| Более 6 лет             | 1 337    |
| Нет опыта               | 7 197    |
| От 3 до 6 лет           | 14 511   |
| От 1 года до 3 лет      | 26 152   |

**Вывод:**
- Основной спрос на специалистов с опытом **от 1 до 3 лет** (53% от всех вакансий).
- Почти **15%** вакансий доступны без опыта — это хорошие точки входа для начинающих специалистов.
- Вакансий с высокими требованиями (более 6 лет) — лишь **2,7%** от базы.
- При рекомендации вакансий необходимо учитывать соответствие уровня опыта пользователя требованиям рынка.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [994]:
# Формируем SQL-запрос для получения топ-5 работодателей по количеству вакансий.
# Здесь мы хотим узнать, какие компании разместили больше всего вакансий на платформе.

query = f'''
select 
	emp.name,         -- Берём название работодателя
	count(*)          -- Считаем, сколько вакансий у каждого работодателя
from public.vacancies as vac
join public.employers as emp
	on vac.employer_id = emp.id  -- Соединяем вакансии с работодателями по id
group by emp.name                -- Группируем по названию работодателя
order by 2 desc                  -- Сортируем по количеству вакансий (от большего к меньшему)
limit 5                          -- Оставляем только топ-5 компаний
'''
# Такой запрос помогает быстро увидеть лидеров рынка по числу открытых позиций.

In [995]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [996]:
# Формируем SQL-запрос для поиска региона, где нет вакансий, но зарегистрировано больше всего работодателей.
# select areas.name — выбираем название региона.
# count(vac.id) as vacancies_count — считаем количество вакансий в регионе (будет 0, т.к. ищем такие регионы).
# count(emp.id) as emp_count — считаем количество работодателей в регионе.
# from areas as areas — работаем с таблицей регионов.
# left join vacancies as vac on areas.id = vac.area_id — присоединяем вакансии к регионам (чтобы видеть, где их нет).
# join employers as emp on areas.id = emp.area — присоединяем работодателей к регионам.
# group by areas.name — группируем по названию региона.
# having count(vac.id) = 0 — оставляем только те регионы, где нет вакансий.
# order by emp_count desc — сортируем по убыванию числа работодателей.
# limit 1 — берём только регион с максимальным числом работодателей.

query = f'''
select 
	areas.name,
	count(vac.id) as vacancies_count,
	count(emp.id) as emp_count
from areas as areas
left join vacancies as vac
on areas.id = vac.area_id
join employers as emp
on areas.id = emp.area
group by areas.name
having count(vac.id) = 0
order by emp_count desc
limit 1
'''

In [997]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,vacancies_count,emp_count
0,Россия,0,410


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [998]:
# Формируем SQL-запрос, чтобы узнать, сколько разных регионов (areas) публикует вакансии каждый работодатель.
# Это поможет понять, насколько широко компании представлены по стране.

query = f'''
select
	emp.name,                             -- Название работодателя
	count(distinct area_id) as areas_count  -- Считаем уникальные регионы, где есть вакансии этого работодателя
from vacancies as vac
left join employers as emp
	on vac.employer_id = emp.id           -- Присоединяем таблицу работодателей по id
group by emp.name                         -- Группируем по названию работодателя
order by 2 desc                           -- Сортируем по убыванию количества регионов
limit 1                                   -- Оставляем только лидера (у кого больше всего регионов)
'''
# В результате получим работодателя, который публикует вакансии в наибольшем числе регионов.

In [999]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,areas_count
0,Яндекс,181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [1000]:
# SQL-запрос для подсчёта количества работодателей без указанной сферы деятельности.
# 1. Выбираем из таблицы employers (работодатели).
# 2. LEFT JOIN соединяет работодателей с таблицей employers_industries (сферы деятельности работодателей).
#    Это нужно, чтобы увидеть всех работодателей, даже если у них нет сферы деятельности.
# 3. В условии WHERE оставляем только тех, у кого нет ни одной сферы (industry_id is null).
# 4. Считаем таких работодателей с помощью count(emp.name).

query = f'''
select
	count(emp.name)
from employers as emp
left join employers_industries as emp_inds
on emp.id = emp_inds.employer_id
where emp_inds.industry_id is null
'''

In [1001]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [1002]:
query = f'''
select
	emp.name,                        -- Берём название работодателя
	count(emp_inds.industry_id)      -- Считаем, сколько сфер деятельности у работодателя
from employers as emp
left join employers_industries as emp_inds
	on emp.id = emp_inds.employer_id -- Присоединяем таблицу сфер деятельности к работодателям
group by emp.name                  -- Группируем по названию работодателя
having count(emp_inds.industry_id) = 4  -- Оставляем только тех, у кого ровно 4 сферы деятельности
order by 1                         -- Сортируем по алфавиту (по названию компании)
offset 2                           -- Пропускаем первые две компании (чтобы получить третью)
limit 1                            -- Берём только одну строку — это и будет нужная компания
'''


In [1003]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [1004]:
# Формируем SQL-запрос для подсчёта количества работодателей, у которых указана сфера деятельности "Разработка программного обеспечения"
query = f'''
select count(employer_id)  -- Считаем количество работодателей (employer_id)
from employers_industries as emp_inds 
join industries as ind
on emp_inds.industry_id = ind.id  -- Соединяем таблицы по id сферы деятельности
where ind.name = 'Разработка программного обеспечения'  -- Оставляем только тех, у кого сфера — разработка ПО
'''
# Этот запрос покажет, сколько компаний на платформе занимаются разработкой программного обеспечения.

In [1005]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [1006]:
from bs4 import BeautifulSoup  # Импортируем библиотеку для парсинга HTML-страниц
import requests  # Импортируем библиотеку для HTTP-запросов

url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'  # Ссылка на страницу с городами-миллионниками
response = requests.get(url)  # Делаем запрос к сайту и получаем HTML-код страницы
soup = BeautifulSoup(response.text, 'html.parser')  # Преобразуем HTML-код в объект BeautifulSoup для удобного поиска

table = soup.find('table')  # Находим первую таблицу на странице (в ней лежит список городов)
rows = table.find_all('tr')  # Получаем все строки таблицы

cities = []  # Создаём пустой список для городов
for row in rows[1:]:  # Пропускаем заголовок таблицы и идём по всем остальным строкам
    # В каждой строке берём все ячейки (td), достаём текст и берём второй столбец — это название города
    city = [td.get_text(strip=True) for td in row.find_all('td')][1]
    if city:  # Если название города не пустое, добавляем его в список
        cities.append(city)
print(cities)  # Выводим итоговый список городов-миллионников

['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Краснодар', 'Самара', 'Ростов-на-Дону', 'Омск', 'Воронеж', 'Пермь', 'Волгоград']


In [1007]:
# Формируем SQL-запрос для получения количества вакансий Яндекса в каждом городе-миллионнике и общей суммы по этим городам

query = f'''
select
	ar.name as area_name,                -- Берём название региона (города)
	count(vac.id) as vacancies_count     -- Считаем количество вакансий Яндекса в этом регионе
from vacancies as vac
join areas as ar
	on vac.area_id = ar.id               -- Присоединяем таблицу регионов к вакансиям
join employers as emp
	on vac.employer_id = emp.id          -- Присоединяем работодателей к вакансиям
where emp.name = 'Яндекс'                -- Оставляем только вакансии Яндекса
  and (ar.name in (
		'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань',
		'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Краснодар',
		'Самара', 'Ростов-на-Дону', 'Омск', 'Воронеж', 'Пермь', 'Волгоград'
	))                                   -- Фильтруем только города-миллионники
group by ar.name                         -- Группируем по названию города
union all                                -- Добавляем к результату ещё одну строку (итоговую сумму)
select
	'total',                             -- В этой строке в качестве названия будет 'total'
	count(vac.id) as total               -- Считаем общее количество вакансий Яндекса по всем этим городам
from vacancies as vac
join areas as ar
	on vac.area_id = ar.id
join employers as emp
	on vac.employer_id = emp.id
where emp.name = 'Яндекс'
  and (ar.name in (
		'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань',
		'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Краснодар',
		'Самара', 'Ростов-на-Дону', 'Омск', 'Воронеж', 'Пермь', 'Волгоград'
	))
order by 2                               -- Сортируем результат по количеству вакансий (по возрастанию)
'''

In [1008]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area_name,vacancies_count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

## Выводы по анализу работодателей

## 1. Крупнейшие работодатели по количеству вакансий

Анализ показал, что лидером по количеству вакансий является компания **Яндекс** с большим отрывом (1933 вакансии), за ней следуют **Ростелеком**, **Тинькофф**, **СБЕР** и **Газпром нефть**. Это указывает на высокую активность крупных технологических и финансовых компаний на рынке труда для специалистов в области Data Science.

**Вывод:**  
Для построения модели рекомендательной системы стоит уделить внимание вакансиям от крупнейших работодателей, так как они обеспечивают основной поток предложений.

---

## 2. Региональное распределение работодателей и вакансий

Несмотря на то, что в некоторых регионах нет ни одной вакансии, работодатели там зарегистрированы.

**Вывод:**  
Следует учитывать, что наличие работодателя в регионе не всегда означает наличие вакансий. Это важно при фильтрации и географической персонализации рекомендаций.

---

## 3. География деятельности работодателей

Наибольшее число регионов с опубликованными вакансиями у компании **Яндекс** — 181 регион. Это говорит о широкой географической представленности и потенциальной гибкости в удалённой работе.

**Вывод:**  
Компании с широкой географией публикации вакансий должны иметь повышенный приоритет при генерации рекомендаций для кандидатов из разных регионов.

---

## 4. Отсутствие указания сферы деятельности

У **8419 работодателей** не указана сфера деятельности, что снижает качество категоризации и фильтрации вакансий.

**Вывод:**  
Необходимо учитывать этот пробел при предобработке данных и рассматривать возможность использования дополнительных источников для заполнения недостающей информации.

---

## 5. Компании с несколькими сферами деятельности

Некоторые компании, например **2ГИС**, указывают сразу четыре сферы деятельности. Такие данные могут быть полезны для определения мультидисциплинарных работодателей.

**Вывод:**  
Компании с несколькими сферами деятельности потенциально предлагают разнообразные и междисциплинарные вакансии, что важно для кандидатов с гибким профилем.

---

## 6. Популярная сфера — Разработка ПО

Сфера "Разработка программного обеспечения" является самой частой — она указана у **3553 работодателей**.

**Вывод:**  
Для позиции Data Scientist важна интеграция с разработкой ПО, поэтому это подтверждает релевантность текущих данных для целевой задачи.

---

## 7. Вакансии Яндекса в городах-миллионниках

Компания **Яндекс** активно публикует вакансии в крупных городах. Наибольшее количество вакансий — в **Москве** и **Санкт-Петербурге**, но также охвачены практически все города-миллионники.

**Вывод:**  
Яндекс предлагает релевантные вакансии в ключевых локациях, что важно для точной геопривязки модели рекомендаций.

---

# Общий вывод

Данные о работодателях содержат полезную информацию для построения рекомендательной системы. Особое внимание следует уделить:

- крупнейшим работодателям;
- географии деятельности;
- качеству заполнения профиля работодателя;
- популярным сферам деятельности.

Эти факторы будут играть ключевую роль в построении качественных рекомендаций для кандидатов на позицию Data Scientist.

In [1009]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# 1. Топ-5 работодателей по количеству вакансий
top_employers = pd.DataFrame({
    'name': ['Яндекс', 'Ростелеком', 'Тинькофф', 'СБЕР', 'Газпром нефть'],
    'count': [1933, 491, 444, 428, 331]
})

fig1 = px.bar(
    top_employers, 
    x='name', 
    y='count',
    title='Топ-5 работодателей по количеству вакансий',
    labels={'name': 'Работодатель', 'count': 'Количество вакансий'}
)
fig1.show()
fig1.write_html("charts/chart1.html")

# 2. Количество работодателей без указания сферы деятельности
missing_industries = pd.DataFrame({
    'category': ['С указанием сферы', 'Без указания сферы'],
    'count': [23501 - 8419, 8419]
})

fig2 = px.pie(
    missing_industries, 
    values='count', 
    names='category',
    title='Доля работодателей без указанной сферы деятельности'
)
fig2.show()
fig2.write_html("charts/chart2.html")

# 3. Количество работодателей по популярной сфере "Разработка ПО"
industry_data = pd.DataFrame({
    'category': ['Разработка ПО', 'Другие сферы'],
    'count': [3553, 23501 - 3553]
})

fig3 = px.pie(
    industry_data, 
    values='count', 
    names='category',
    title='Работодатели в сфере "Разработка ПО"'
)
fig3.show()
fig3.write_html("charts/chart3.html")

# 4. География вакансий Яндекса в городах-миллионниках
yandex_regions = pd.DataFrame({
    'area_name': [
        'Омск', 'Челябинск', 'Красноярск', 'Волгоград', 'Пермь', 'Казань',
        'Ростов-на-Дону', 'Уфа', 'Самара', 'Краснодар', 'Воронеж',
        'Новосибирск', 'Нижний Новгород', 'Екатеринбург', 'Санкт-Петербург', 'Москва'
    ],
    'vacancies_count': [21, 22, 23, 24, 25, 25, 25, 26, 26, 30, 32, 35, 36, 39, 42, 54]
})

fig4 = px.bar(
    yandex_regions, 
    x='area_name', 
    y='vacancies_count',
    title='Вакансии Яндекса в городах-миллионниках',
    labels={'area_name': 'Город', 'vacancies_count': 'Количество вакансий'},
    color='vacancies_count'
)
fig4.update_layout(xaxis_tickangle=-45)
fig4.show()
fig4.write_html("charts/chart4.html")

# 5. Количество регионов, в которых публикует вакансии Яндекс
fig5 = go.Figure(go.Indicator(
    mode="number",
    value=181,
    title={"text": "Количество регионов, в которых Яндекс публикует вакансии"}
))
fig5.show()
fig5.write_html("charts/chart5.html")


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [1010]:
# Формируем SQL-запрос для подсчёта количества вакансий, связанных с данными.
# lower(vac.name) приводит название вакансии к нижнему регистру — так мы не пропустим варианты с разным написанием.
# like '%data%' ищет в названии слово "data" в любом месте строки.
# like '%данн%' ищет в названии подстроку "данн" (например, "данные", "данных" и т.д.).
# В итоге считаем все вакансии, где в названии встречается "data" или "данн".

query = f'''
select count(*)
from vacancies as vac
where lower(vac.name) like '%data%' or lower(vac.name) like '%данн%'
'''

In [1011]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [1012]:
# Формируем SQL-запрос, чтобы узнать, сколько вакансий для начинающих дата-сайентистов
query = f'''
SELECT COUNT(*)
FROM vacancies AS vac
WHERE (
    -- Проверяем, что вакансия относится к Data Science (ищем разные варианты в названии)
    lower(vac.name) LIKE '%data scientist%' OR 
    lower(vac.name) LIKE '%data science%' OR
    lower(vac.name) LIKE '%исследователь данных%' OR
    (lower(vac.name) LIKE '%ml%' AND lower(vac.name) NOT LIKE '%html%') OR  -- ML, но не HTML
    lower(vac.name) LIKE '%machine learning%' OR
    lower(vac.name) LIKE '%машинн%обучен%'
)
AND (
    -- Считаем вакансию "джуниорской", если в названии есть "junior",
    lower(vac.name) LIKE '%junior%' OR
    -- или если не требуется опыт,
    vac.experience = 'Нет опыта' OR
    -- или если это стажировка
    vac.employment = 'Стажировка'
);
'''

In [1013]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [1014]:
# Формируем SQL-запрос для подсчёта количества вакансий для Data Science, где среди ключевых навыков указан SQL или postgres

query = f'''
SELECT COUNT(*)
FROM vacancies AS vac
WHERE (
    -- Проверяем, что вакансия относится к Data Science (ищем разные варианты в названии)
    lower(vac.name) LIKE '%data scientist%' OR 
    lower(vac.name) LIKE '%data science%' OR
    lower(vac.name) LIKE '%исследователь данных%' OR
    (lower(vac.name) LIKE '%ml%' AND lower(vac.name) NOT LIKE '%html%') OR  -- ML, но не HTML
    lower(vac.name) LIKE '%machine learning%' OR
    lower(vac.name) LIKE '%машинн%обучен%'
)
AND (
    -- Смотрим, чтобы в ключевых навыках был SQL или postgres
    vac.key_skills LIKE '%SQL%' OR
    vac.key_skills LIKE '%postgres%'
);
'''

In [1015]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [1016]:
# Формируем SQL-запрос, чтобы узнать, сколько вакансий для Data Science требуют знание Python
query = f'''
SELECT COUNT(*)
FROM vacancies AS vac
WHERE (
    -- Проверяем, что вакансия относится к Data Science по ключевым словам в названии
    lower(vac.name) LIKE '%data scientist%' OR 
    lower(vac.name) LIKE '%data science%' OR
    lower(vac.name) LIKE '%исследователь данных%' OR
    (lower(vac.name) LIKE '%ml%' AND lower(vac.name) NOT LIKE '%html%') OR  -- ищем ML, но не HTML
    lower(vac.name) LIKE '%machine learning%' OR
    lower(vac.name) LIKE '%машинн%обучен%'
)
AND (
    -- Смотрим, чтобы среди ключевых навыков был Python
    vac.key_skills LIKE '%Python%'
);
'''
# Этот запрос посчитает, в скольких вакансиях для DS среди требований явно указан Python.

In [1017]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [1018]:
# SQL-запрос для подсчёта среднего количества ключевых навыков в вакансиях для Data Scientist
# 
# 1. Считаем, сколько раз встречается символ табуляции (CHR(9)) в поле key_skills — это разделитель между навыками.
# 2. LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), '')) — так узнаём, сколько табуляций в строке (то есть сколько разделителей между навыками).
# 3. Прибавляем 1 — получаем общее количество навыков (например, если 2 табуляции, значит 3 навыка).
# 4. AVG(...) — берём среднее по всем подходящим вакансиям.
# 5. ROUND(..., 2) — округляем результат до двух знаков после запятой для красоты.
# 6. В WHERE фильтруем только те вакансии, которые относятся к Data Science (по ключевым словам в названии) и у которых заполнено поле key_skills.

query = f'''
SELECT
    ROUND(AVG(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), '')) + 1), 2)
FROM vacancies v
WHERE (
    v.name ILIKE '%data scientist%' OR
    v.name ILIKE '%data science%' OR
    v.name ILIKE '%исследователь данных%' OR
    (v.name LIKE '%ML%' AND v.name NOT ILIKE '%html%') OR
    v.name ILIKE '%machine learning%' OR
    v.name ILIKE '%машинн%обучен%'
)
AND key_skills IS NOT NULL
'''

In [1019]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [1020]:
# Формируем SQL-запрос, чтобы узнать среднюю зарплату для DS по каждому типу опыта.
# В этом запросе:
# 1. Внутренний подзапрос sub_table:
#    - Для каждой вакансии считаем среднюю зарплату:
#      - Если указаны обе границы (salary_from и salary_to), берём их среднее.
#      - Если указана только одна граница, берём её как зарплату.
#      - COALESCE помогает не получить null, если одно из полей не заполнено.
#    - Группируем по id вакансии.
# 2. Внешний запрос:
#    - Присоединяем к основной таблице вакансий наш подзапрос по id.
#    - Фильтруем только те вакансии, которые относятся к Data Science (по ключевым словам в названии).
#    - Для каждого варианта опыта (experience) считаем среднюю зарплату (округляем до целого).
#    - Группируем по опыту, чтобы получить отдельную строку для каждого уровня.

query = f'''
SELECT
    v.experience,                          -- Тип требуемого опыта (например, "Нет опыта", "От 3 до 6 лет" и т.д.)
    ROUND(AVG(sub_table.mean)) AS avg_salary  -- Средняя зарплата для этого опыта (округляем до целого)
FROM vacancies v
JOIN (
    SELECT
        v.id,
        AVG((
            COALESCE(v.salary_from, v.salary_to) + 
            COALESCE(v.salary_to, v.salary_from)
        ) / 2) AS mean  -- Считаем среднюю зарплату по вакансии (или одну из границ, если вторая не указана)
    FROM vacancies v
    WHERE v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL  -- Берём только вакансии, где указана хотя бы одна граница зарплаты
    GROUP BY v.id, v.salary_from, v.salary_to
) AS sub_table ON v.id = sub_table.id
WHERE (
    LOWER(v.name) LIKE '%data scientist%' OR
    LOWER(v.name) LIKE '%data science%' OR
    LOWER(v.name) LIKE '%исследователь данных%' OR
    (v.name LIKE '%ML%' AND v.name NOT ILIKE '%html%') OR
    LOWER(v.name) LIKE '%machine learning%' OR
    LOWER(v.name) LIKE '%машинн%обучен%'
)  -- Фильтруем только вакансии, связанные с Data Science
GROUP BY v.experience;  -- Группируем по опыту, чтобы получить среднюю зарплату для каждого уровня'''

In [1021]:
df = pd.read_sql_query(query, connection)
df

/var/folders/qp/2ywt8815723ds834k_r7cwkr0000gn/T/ipykernel_46210/137330334.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

## Выводы по анализу вакансий для Data Scientist

## 1. Количество вакансий, связанных с данными

По ключевым словам 'data' или 'данн' в названии вакансии найдено **1771 вакансия**. Это подтверждает наличие значительного объёма предложений, относящихся к работе с данными.

**Вывод:**  
Рынок вакансий, связанных с анализом и обработкой данных, достаточно велик, что делает задачу построения рекомендательной системы актуальной.

---

## 2. Количество вакансий для начинающих дата-сайентистов

По набору ключевых слов, соответствующих профессии Data Scientist, и фильтрам по опыту и типу занятости, найдено **51 вакансия**, подходящая для начинающих специалистов.

**Вывод:**  
Для Junior-специалистов в области Data Science вакансий значительно меньше, чем общего объёма, что необходимо учитывать при построении индивидуальных рекомендаций для соискателей без опыта.

---

## 3. Популярность SQL и Postgres среди требований к DS

SQL или Postgres указаны как ключевые навыки в **229 вакансиях** для дата-сайентистов.

**Вывод:**  
Базы данных и навыки работы с SQL являются одним из основных требований к специалистам по данным, что следует учитывать в профиле компетенций кандидата.

---

## 4. Популярность Python

Python указан как ключевой навык в **357 вакансиях** для DS.

**Вывод:**  
Python — доминирующий язык программирования в сфере Data Science. Учитывая его популярность, он должен быть приоритетным в профиле соискателя и модели рекомендаций.

---

## 5. Среднее количество ключевых навыков в вакансиях для DS

В среднем, в вакансиях для дата-сайентистов указывается **6.41 ключевых навыка**.

**Вывод:**  
Вакансии часто требуют широкий спектр навыков, поэтому полезно учитывать как основные, так и дополнительные компетенции кандидата при подборе вакансий.

---

## 6. Средняя зарплата в зависимости от опыта

| Требуемый опыт          | Средняя зарплата |
|-------------------------|------------------|
| Нет опыта               | 74 643           |
| От 1 года до 3 лет      | 139 675          |
| От 3 до 6 лет           | 243 115          |

**Вывод:**  
С увеличением опыта наблюдается существенный рост зарплаты. Это важно для прогноза уровня дохода кандидата и построения карьерной траектории. Модель может использовать эти данные для фильтрации и приоритизации вакансий по ожидаемой зарплате.

---

# Общий вывод

На основании анализа вакансий, связанных с Data Science, можно сделать следующие ключевые заключения:

- Имеется значительный объём вакансий, связанных с данными, однако предложения для начинающих специалистов ограничены.
- Основные востребованные навыки: Python, SQL, машинное обучение.
- Требования к кандидатам многогранны, в среднем указывается более 6 навыков.
- Уровень зарплаты напрямую зависит от опыта, что важно учитывать при персонализации предложений.

Эти выводы подтверждают релевантность данных для задачи построения рекомендательной системы и задают направления для инженерии признаков в будущей модели.

In [1022]:
# 1. Количество вакансий, связанных с данными
data_vacancies = pd.DataFrame({
    'Категория': ['Вакансии с "данн"/"data"', 'Прочие'],
    'Количество': [1771, 49197 - 1771]
})
fig1 = px.pie(data_vacancies, names='Категория', values='Количество',
              title='Доля вакансий, связанных с данными')
fig1.show()
fig1.write_html("charts/chart6.html")

# 2. Количество вакансий для Junior DS
junior_ds_vacancies = pd.DataFrame({
    'Категория': ['Junior DS вакансии', 'Прочие DS вакансии'],
    'Количество': [51, 1771 - 51]
})
fig2 = px.bar(junior_ds_vacancies, x='Категория', y='Количество',
              title='Вакансии для начинающих дата-сайентистов')
fig2.show()
fig2.write_html("charts/chart7.html")

# 3. Популярность навыков: SQL/Postgres и Python
skills = pd.DataFrame({
    'Навык': ['SQL/Postgres', 'Python'],
    'Количество': [229, 357]
})
fig3 = px.bar(skills, x='Навык', y='Количество',
              title='Популярность навыков в вакансиях DS',
              color='Навык')
fig3.show()
fig3.write_html("charts/chart8.html")

# 4. Среднее количество навыков в DS вакансиях
fig4 = go.Figure(go.Indicator(
    mode="number",
    value=6.41,
    title={"text": "Среднее число ключевых навыков в DS вакансиях"}
))
fig4.show()
fig4.write_html("charts/chart9.html")

# 5. Средняя зарплата по опыту
salary_data = pd.DataFrame({
    'Опыт': ['Нет опыта', '1-3 года', '3-6 лет'],
    'Средняя зарплата': [74643, 139675, 243115]
})
fig5 = px.bar(salary_data, x='Опыт', y='Средняя зарплата',
              title='Средняя зарплата по уровню опыта (DS вакансии)',
              text='Средняя зарплата')
fig5.update_traces(textposition='outside')
fig5.show()
fig5.write_html("charts/chart10.html")

# Общий вывод по проекту

## Итоговое исследование: Анализ вакансий Data Scientist на hh.ru

## Цель проекта

Разработка рекомендательной системы для подбора вакансий на позицию Data Scientist на основе данных с платформы hh.ru. В фокусе — анализ вакансий, требований, работодателей и географии.

---

## 1. Общие характеристики данных

| Показатель                  | Значение      |
|-----------------------------|---------------|
| Количество вакансий         | 49 197        |
| Количество работодателей    | 23 501        |
| Количество регионов         | 1 362         |
| Количество сфер деятельности| 294           |

**Вывод:**
- Данные объемные, разнообразные и пригодны для построения ML-модели.
- Присутствует высокая географическая и профессиональная вариативность, что положительно влияет на универсальность и масштабируемость рекомендаций.

---

## 2. Вакансии, связанные с Data Science

- По ключевым словам "данн" или "data" в названии найдено **1771 вакансия**.
- Среди них только **51 вакансия** подходит начинающим специалистам.

**Вывод:**
- Рынок вакансий, связанных с анализом данных, достаточно развит.
- Однако для Junior-специалистов вакансий существенно меньше, что требует особого подхода при построении персонализированных рекомендаций.

---

## 3. Требуемые навыки

- SQL/Postgres: упоминается в **229** вакансиях DS.
- Python: упоминается в **357** вакансиях DS.
- В среднем указано **6.41** ключевых навыков на вакансию.

**Вывод:**
- Python и SQL — основные технологии в сфере Data Science.
- Кандидаты должны обладать широким спектром компетенций, что важно учитывать при сравнении профиля пользователя и требований вакансий.

---

## 4. Зарплата и опыт

| Требуемый опыт          | Средняя зарплата |
|-------------------------|------------------|
| Нет опыта               | 74 643           |
| От 1 до 3 лет           | 139 675          |
| От 3 до 6 лет           | 243 115          |

**Вывод:**
- Наблюдается стабильный рост зарплаты с увеличением опыта.
- Эти данные можно использовать для прогнозирования дохода и определения релевантных вакансий под уровень кандидата.

---

## 5. География и график работы

- Лидеры по числу вакансий: Москва (5 333), Санкт-Петербург (2 851), Минск, Новосибирск, Алматы.
- Популярный формат: **Полный день + Полная занятость** — 72% вакансий.
- Удалённая работа — уверенно занимает второе место по популярности.

**Вывод:**
- Рынок сконцентрирован в мегаполисах, но активно развивается и в других регионах.
- Учитывая популярность удалённой работы, необходимо включать соответствующие фильтры в рекомендательную систему.

---

## 6. Опыт и распределение по нему

| Требуемый опыт     | Количество вакансий |
|--------------------|----------------------|
| Нет опыта          | 7 197                |
| 1–3 года           | 26 152               |
| 3–6 лет            | 14 511               |
| Более 6 лет        | 1 337                |

**Вывод:**
- Основной спрос — на специалистов с 1–3 годами опыта (53%).
- Почти 15% вакансий доступны без опыта, что делает возможным построение трека входа в профессию.

---

## 7. Анализ работодателей

- Лидеры по числу вакансий: Яндекс (1933), Ростелеком, Тинькофф, СБЕР, Газпром нефть.
- У Яндекса — наибольшая география охвата (181 регион).
- Часто отсутствует указание сферы деятельности (у 8 419 компаний).

**Выводы:**
- Важно учитывать профиль и активность работодателя в модели.
- География работодателя может стать дополнительным параметром для фильтрации.
- Мультидисциплинарные компании требуют отдельного подхода в анализе.

---

## 8. Дополнительные предложения для развития исследования

### 8.1. Прогнозирование зарплаты
- Использовать модели регрессии для прогнозирования зарплатной вилки на основе требований вакансий.

### 8.2. Кластеризация вакансий
- Применить алгоритмы кластеризации для группировки вакансий по типовым профилям: Data Analyst, ML Engineer, Research Scientist и др.

### 8.3. Анализ тенденций во времени
- Если доступны временные метки — оценить, как меняется спрос на DS-специалистов в динамике.

### 8.4. Пополнение недостающих данных
- Импутировать пропущенные значения (например, сферу деятельности) с помощью моделей на основе текстов вакансий и профилей работодателей.

### 8.5. Построение карьерной траектории
- На основе анализа опыта, навыков и зарплат — построить вероятностную модель развития карьеры DS-специалиста.

---

## Общий итог

На основании собранных и проанализированных данных можно утверждать, что:

- Существуют объективные основания для построения ML-модели рекомендаций для Data Scientist.
- Рынок разнообразен и насыщен, особенно в городах-миллионниках и у крупных работодателей.
- Требования к кандидатам ясно очерчены, что даёт возможность точной фильтрации и персонализации.
- При проектировании системы важно учитывать ограничения для начинающих, пропущенные значения и предпочтения по локации и графику.

Дальнейшие шаги — формализация признаков, разработка прототипа модели и валидация качества рекомендаций на тестовой выборке.